# PYSPARK - PRÁCTICA 3 (NBA)

#### Álvaro Rodríguez López

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

**Persist y cache** son mecanismos nativos de almacenamiento que optimizan las operaciones en spark. Ambas
ayudan salvando los RDDS en memoria cache para ser utilizados en fases posteriores. Son operaciones de tipo
transformación.


Las diferencias entre las dos técnicas que acabamos de comentar son que mientras con **cache()** se utliza el
nivel de memoria predeterminado, con **persist()** se puede especificar el nivel de memoria a utilizar.


Este tipo de mecanismos son efectivos para guardar pequeños conjuntos de datos, así como en canalizaciones
ETL donde tenga que almacenar en caché los resultados intermedios, pero no se recomiendan para la creación
de particiones.


La función **collect()**, de tipo acción, devuelve al ejecutarla todos los elementos de un RDD. Cuando tenemos una
enorme cantidad de datos, realizamos transformaciones, filtros por ejemplo, y, una vez realizados, nos traemos
ese resultado como un array usando la función collect().


## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

Para llevar acabo la carga de datos podemos realizar las siguientes tareas:
    
  - inicializar spark context.
    
  - Importar los datos en spark context.
    
  - Comprobar el número de registros a importar.
    
  - Definir expresiones regulares para el parseado de los datos. Comprobar que las expresiones regulares coinciden con las distintas lineas.
    
  - Parsear conforme a la expresiones regulares chequeadas anteriormente.
    
  - Una vez paresados los datos, comprobar que el número de registros importados coincide.
    
  - Definir los tipos apropiados y las funciones que puedan ser precisas para tratar cada una de las partes parseadas.

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

   
 - Flujos de información no protocolizados: La falta de una estrategia clara en la gestión de información puede conllevar la rectificación de variables innecesarias(**determinar las variables que hay que eliminar**), la gestión de bases de datos con valores vacios (**definir la elminación, imputación o conservación de valores vacíos en función de cada variable**), la posibilidad de encontrar valores duplicados (**definir el tratamiento de valores duplicados para las distintas variables, su localización y su posterior eliminación o corrección**) y por último, la posibilidad de encontrar erratas en la imputación (**localizar valores no matcheables con expresiones regulares y determinar su eliminación o imputación**).
 
 
 - Falta de coordinación entre el área de datos y las áreas de data science o BI: Algunas variables de información que podrían resultar muy interesantes para departamentos de investigación y reporting no se integran en el flujo de información de la empresa. Puede suponer una perdida de oportunidades aunque siempre es posible **definir la creación de nuevas variables** en función de las necesidades concretas y las variables disponibles.

- Falta de información sobre negocio: Finalmente los datos utilizados deben comprobarse en su conjunto para determinar si la información se corresponde con la realidad (Ej: Comparamos registros de nuevos clientes por meses y comprobamos que hay una subida en diciembre, consistente con lo que sabemos sobre el negocio de nuestra compañía). Es por ello que **llevar un registro de subidas** nos puede ayudar a identificar otros problemas.

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

- **Inicializamos SparkContext**

In [1]:
from pyspark import SparkContext

sc = SparkContext()


In [366]:
# Otros paquetes
import time  
import re
from datetime import datetime
from datetime import date, timedelta

- **Importamos data**

In [224]:
data_file = "./partidosLigaNBA.csv"
raw_data = sc.textFile(data_file)

In [225]:
raw_data.take(5)

['Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1',
 'Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93',
 'Tue, Oct 30, 2007:"7:00 pm":Portland Trail Blazers:97:San Antonio Spurs:106',
 'Wed, Oct 31, 2007:"8:00 pm":Dallas Mavericks:92:Cleveland Cavaliers:74']

In [226]:
raw_data.count()

12908

- **Parseado de raw data**

1) Eliminamos el header

In [235]:
header = raw_data.first() 
raw_data_t1 = raw_data.filter(lambda linea: linea != header)

In [249]:
raw_data_t1.take(5)

['Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93',
 'Tue, Oct 30, 2007:"7:00 pm":Portland Trail Blazers:97:San Antonio Spurs:106',
 'Wed, Oct 31, 2007:"8:00 pm":Dallas Mavericks:92:Cleveland Cavaliers:74',
 'Wed, Oct 31, 2007:"8:30 pm":Seattle SuperSonics:103:Denver Nuggets:120']

In [248]:
raw_data_t1.count() 

12907

2) Eliminamos las filas que no aportan info

In [577]:
raw_data_t1.filter(lambda x: "Playoffs" in x).take(1) # comprobamos el aspecto de estas filas

['Playoffs:Playoffs:Playoffs:Playoffs:Playoffs:Playoffs']

In [272]:
raw_data_t1.filter(lambda x: "Playoffs" in x).count() # comprobamos el número de filas que hay que eliminar.

10

In [270]:
raw_data_t2 = raw_data_t1.filter(lambda x: "Playoffs" not in x) # Elimino las 10 filas que cotienen Playoffs

In [271]:
raw_data_t2.filter(lambda x: "Playoffs" in x).count() # Compruebo que no quedan ninguna

0

In [579]:
raw_data_t2.count() # Compruebo que ha sido eliminadas

12897

3) Split (dividimos por ":")

In [580]:
parsed_data = raw_data_t2.map(lambda linea: linea.split(":"))

In [581]:
parsed_data.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

4) Formato fecha

In [583]:
# Función de conversion del tipo de fecha            

# Year
def year(linea):
    from_date = linea[0]
    conv=time.strptime(from_date,"%a, %b %d, %Y")
    return time.strftime("%Y",conv)

# Month

def month(linea):
    from_date = linea[0]
    conv=time.strptime(from_date,"%a, %b %d, %Y")
    return time.strftime("%m",conv)

# consideramos cada temporada por el año en que termina. Ej: Temporada 2017, comienza en 2016 y termina en 2017.

def season(linea):
    from_date = linea[0]
    conv=time.strptime(from_date,"%a, %b %d, %Y")
    month = conv.tm_mon
    if month < 8:
            return conv.tm_year
    else:
            return conv.tm_year + 1

# Ganador
def ganador(linea):
    if linea[4] > linea[6]:
            return linea[3]
    else:
            return linea[5]


## Quinta tarea: Media de la diferencia de puntos por año

In [594]:
# diferencia acumulada de puntos en cada partido.

diff_year = (parsed_data.map(lambda linea: (year(linea), abs(int(linea[4]) - int(linea[6]))))
                       .reduceByKey(lambda a,b: a+b))

# Partidos en cada año

matches_year = (parsed_data.map(lambda linea: (year(linea),1))
                       .reduceByKey(lambda a,b: a+b))

# Media de la diferencia de puntos por cada año
diff_year_mean =  (diff_year.join(matches_year)
                         .map(lambda linea: (linea[0], round((linea[1][0])/(linea[1][1]),3))))               
               
               

In [595]:
diff_year_mean.takeOrdered(11, lambda x: -x[1]) # años ordenados por media de diff de puntos.

[('2016', 11.551),
 ('2008', 11.544),
 ('2017', 11.422),
 ('2015', 11.16),
 ('2007', 11.096),
 ('2009', 11.09),
 ('2013', 11.072),
 ('2014', 10.905),
 ('2010', 10.869),
 ('2012', 10.845),
 ('2011', 10.661)]

## Sexta tarea: ¿Han jugado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

In [293]:
# partidos de cada equipo como local
matches_team_local = (parsed_data.map(lambda linea: (linea[3],1))
                       .reduceByKey(lambda a,b: a+b))

# partidos de cada equipo como visitante
matches_team_visitante = (parsed_data.map(lambda linea: (linea[5],1))
                       .reduceByKey(lambda a,b: a+b))

# partido de cada equipo totales
matches_team_total =  (matches_team_local.join(matches_team_visitante)
                         .map(lambda a: (a[0],(a[1][0])+(a[1][1]))))               
               

In [297]:
matches_team_total.takeOrdered(50, lambda x: -x[1])

[('San Antonio Spurs', 933),
 ('Boston Celtics', 930),
 ('Miami Heat', 917),
 ('Cleveland Cavaliers', 901),
 ('Los Angeles Lakers', 897),
 ('Atlanta Hawks', 896),
 ('Golden State Warriors', 885),
 ('Chicago Bulls', 873),
 ('Houston Rockets', 868),
 ('Indiana Pacers', 868),
 ('Memphis Grizzlies', 867),
 ('Dallas Mavericks', 867),
 ('Orlando Magic', 863),
 ('Los Angeles Clippers', 861),
 ('Portland Trail Blazers', 853),
 ('Toronto Raptors', 850),
 ('Denver Nuggets', 848),
 ('Utah Jazz', 846),
 ('Washington Wizards', 844),
 ('Philadelphia 76ers', 834),
 ('Detroit Pistons', 829),
 ('Milwaukee Bucks', 827),
 ('Phoenix Suns', 825),
 ('New York Knicks', 825),
 ('Oklahoma City Thunder', 818),
 ('Minnesota Timberwolves', 804),
 ('Sacramento Kings', 804),
 ('Charlotte Bobcats', 566),
 ('New Orleans Hornets', 499),
 ('Brooklyn Nets', 435),
 ('New Jersey Nets', 394),
 ('New Orleans Pelicans', 332),
 ('Charlotte Hornets', 253),
 ('Seattle SuperSonics', 82)]

Los  equipos han jugado distinto número de partidos.

- Algunas franquicias, las que menos partidos han disputado en los años de la base de datos, desaparecieron o se crearon estos años y no disputaron todas las temporadas tenidas en consideración.

- Las diferencias entre las franquicias que han disputado todas las temporadas puede deberse a los playoff. No todos los equipos lo juegan, pueden disputar distintas rondas y distinto número de partido en cada ronda.

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Cleveland?

In [632]:
# Solución 1
# Partidos locales de los cavaliers, ganados, en el mes de enero de cualquier año.

local_cleveland_jan = (parsed_data.filter(lambda linea: linea[3] == "Cleveland Cavaliers")
                       .filter(lambda linea: int(linea[4]) > int(linea[6]))
                       .filter(lambda linea: month(linea) == "01")
                       .map(lambda linea: (month(linea),1))
                       .reduceByKey(lambda a,b: a+b))

# Partidos visitantes de los cavaliers, ganados, en el mes de enero de cualquier año.

visit_cleveland_jan = (parsed_data.filter(lambda linea: linea[5] == "Cleveland Cavaliers")
                       .filter(lambda linea: int(linea[6]) > int(linea[4]))
                       .filter(lambda linea: month(linea) == "01")
                       .map(lambda linea: (month(linea),1))
                       .reduceByKey(lambda a,b: a+b))

# Partidos totales ganados en el mes de enero de cualquier año.
wins_cleveland_jan =  (local_cleveland_jan.join(visit_cleveland_jan)
                         .map(lambda a: (a[0], (a[1][0])+(a[1][1]))))               
               

In [637]:
# Solución 2

# Partidos jugados en enero, ganados por cleveland 

cleveland_jan_wins = (parsed_data.filter(lambda linea: month(linea) == "01")
         .filter(lambda linea: ganador(linea) == "Cleveland Cavaliers"))

In [638]:
cleveland_jan_wins.count()

93

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

In [357]:
# Local
#Partidos ganados
win_local_warriors = (parsed_data.filter(lambda linea: linea[3] == "Golden State Warriors")
                     .filter(lambda linea: int(linea[4]) > int(linea[6]))
                     .count())
# Partidos jugados                 
matches_local_warriors = (parsed_data.filter(lambda linea: linea[3] == "Golden State Warriors")
                     .count())
  
# Visitante
# ganados
win_visit_warriors = (parsed_data.filter(lambda linea: linea[5] == "Golden State Warriors")
                     .filter(lambda linea: int(linea[6]) > int(linea[4]))
                     .count())
# jugados                    
matches_visit_warriors = (parsed_data.filter(lambda linea: linea[5] == "Golden State Warriors")
                     .count())
    
# Ratios

local = win_local_warriors / matches_local_warriors

visit = win_visit_warriors / matches_visit_warriors

print("Los warriors han conseguido un porcentaje de victorias como locales de {}, habiendo ganado {} de {} partidos."
.format(round(local,3),win_local_warriors,matches_local_warriors))


print("Los warriors han conseguido un porcentaje de victorias como visitantes de {}, habiendo ganado {} de {} partidos."
.format(round(visit,3),win_visit_warriors,matches_visit_warriors))

# Son mejores como visitantes.

Los warriors han conseguido un porcentaje de victorias como locales de 0.489, habiendo ganado 215 de 440 partidos.
Los warriors han conseguido un porcentaje de victorias como visitantes de 0.692, habiendo ganado 308 de 445 partidos.


## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

In [663]:
# victorial locales de cada equipo por temporada

win_local_season = (parsed_data.filter(lambda linea: int(linea[4]) > int(linea[6]))
                    .map(lambda linea:((season(linea),linea[3]),1))
                    .reduceByKey(lambda a,b: a+b))
             

win_visit_season = (parsed_data.filter(lambda linea: int(linea[6]) > int(linea[4]))
                    .map(lambda linea:((season(linea),linea[5]),1))
                    .reduceByKey(add))
        

wins_max_season =  (win_local_season.join(win_visit_season)
                      .map(lambda linea:(linea[0][0],(linea[0][1],linea[1][0]+linea[1][1])))
                      .groupByKey()
                      .map(lambda x : (x[0], max(list(x[1]), key=lambda item: item[1]))))

In [665]:
wins_max_season.collect()

[(2016, ('Golden State Warriors', 88)),
 (2008, ('Boston Celtics', 82)),
 (2010, ('Los Angeles Lakers', 73)),
 (2012, ('Miami Heat', 62)),
 (2014, ('San Antonio Spurs', 78)),
 (2017, ('Golden State Warriors', 83)),
 (2011, ('Dallas Mavericks', 73)),
 (2013, ('Miami Heat', 82)),
 (2015, ('Golden State Warriors', 83)),
 (2009, ('Los Angeles Lakers', 81))]

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

In [683]:
# telefonos

texto = "Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com"

# definimos la cadena de expresiones
regexp_tel = '(?:(\d+) ?\(?(\d{2})\)?[\s+\.]?)?(\d{2})[\s+\.]?(\d{2})' # telefono
regexp_correo = "(\S+@\S+)" # correo

telefono = re.findall(regexp_tel,texto)
correo = re.findall(regexp_correo,texto)

print(telefono,correo)

[('902', '13', '23', '13'), ('902', '88', '30', '08'), ('902', '88', '88', '35')] ['atencioncliente@bankinter.com']
